In [1]:
import csv
import sys
import logging
import os
import gensim
from gensim.models import Doc2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from gensim import corpora
import json
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
with open('movie_wordlist.json', 'r') as f:
    data = json.load(f)

D:\Anaconda\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
movie_title = data['movie_title']
movie_content = data['movie_content']

In [3]:
user_text = []
f = open('user_data.txt', encoding='utf-8')
for i in f:
    i=i.strip().split('\n')
    user_text.extend(i)

In [4]:
#similarity between user_text and movie content
texts = movie_content
dictionary = corpora.Dictionary(texts)
dictionary.save('movie_words.dict')
#print(dictionary.token2id)

2018-08-08 21:20:36,514 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-08-08 21:20:36,517 : INFO : built Dictionary(481 unique tokens: ['academy', 'action', 'african', 'all-time', 'america']...) from 3 documents (total 2226 corpus positions)
2018-08-08 21:20:36,518 : INFO : saving Dictionary object under movie_words.dict, separately None
2018-08-08 21:20:36,520 : INFO : saved movie_words.dict


In [5]:
new_doc = user_text
new_vec = dictionary.doc2bow(user_text)
#print(new_vec)

In [8]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize('m_text.mm', corpus)
#print(corpus)

2018-08-08 21:21:23,056 : INFO : storing corpus in Matrix Market format to m_text.mm
2018-08-08 21:21:23,058 : INFO : saving sparse matrix to m_text.mm
2018-08-08 21:21:23,059 : INFO : PROGRESS: saving document #0
2018-08-08 21:21:23,060 : INFO : saved 3x481 matrix, density=37.976% (548/1443)
2018-08-08 21:21:23,062 : INFO : saving MmCorpus index to m_text.mm.index


In [16]:
from gensim import corpora, models, similarities
corpus = corpora.MmCorpus('m_text.mm')

tfidf = models.TfidfModel(corpus)
doc_bow = new_vec
corpus_tfidf = tfidf[corpus]

lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=5)
corpus_lsi = lsi[corpus_tfidf] #bow->tfidf->fold-in-lsi
vec_bow = new_vec
vec_lsi = lsi[vec_bow]

2018-08-08 21:42:54,468 : INFO : loaded corpus index from m_text.mm.index
2018-08-08 21:42:54,470 : INFO : initializing cython corpus reader from m_text.mm
2018-08-08 21:42:54,471 : INFO : accepted corpus with 3 documents, 481 features, 548 non-zero entries
2018-08-08 21:42:54,472 : INFO : collecting document frequencies
2018-08-08 21:42:54,473 : INFO : PROGRESS: processing document #0
2018-08-08 21:42:54,474 : INFO : calculating IDF weights for 3 documents and 480 features (548 matrix non-zeros)
2018-08-08 21:42:54,477 : INFO : using serial LSI version on this node
2018-08-08 21:42:54,477 : INFO : updating model with new documents
2018-08-08 21:42:54,480 : INFO : preparing a new chunk of documents
2018-08-08 21:42:54,481 : INFO : using 100 extra samples and 2 power iterations
2018-08-08 21:42:54,482 : INFO : 1st phase: constructing (481, 105) action matrix
2018-08-08 21:42:54,483 : INFO : orthonormalizing (481, 105) action matrix
2018-08-08 21:42:54,495 : INFO : 2nd phase: running den

In [30]:
index = similarities.MatrixSimilarity(lsi[corpus])
sims = index[vec_lsi]
#print(list(enumerate(sims)))
similarity = {}
sim_id = []
sim_rate = []
for i, element in enumerate(sims):
    sim_id.append(i+1)
    sim_rate.append(element)
similarity = dict(zip(sim_id, sim_rate))
similarity = sorted(similarity.items(), key=lambda item:item[1], reverse=True)
print(similarity)

2018-08-09 01:29:21,793 : WARNING : scanning corpus to determine the number of features (consider setting `num_features` explicitly)
2018-08-09 01:29:21,795 : INFO : creating matrix with 3 documents and 3 features


[(3, 0.8632605), (1, 0.44300938), (2, 0.35772368)]


D:\Anaconda\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
